In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/iitgai-ovha-23/sample_submission.csv
/kaggle/input/iitgai-ovha-23/train_labels.csv
/kaggle/input/iitgai-ovha-23/images/medicakolkata_ccu1_mon--3_2022_6_20_21_0_26.jpeg
/kaggle/input/iitgai-ovha-23/images/medicakolkata_ccu3_mon--1_2022_5_30_15_2_26.jpeg
/kaggle/input/iitgai-ovha-23/images/medicakolkata_ccu3_mon--6_2022_6_8_8_3_31.jpeg
/kaggle/input/iitgai-ovha-23/images/medicakolkata_ccu2_mon--1_2022_6_8_23_6_21.jpeg
/kaggle/input/iitgai-ovha-23/images/medicakolkata_ccu2_mon--6_2022_6_8_9_7_13.jpeg
/kaggle/input/iitgai-ovha-23/images/medicakolkata_rcu_mon--8_2022_5_30_10_5_26.jpeg
/kaggle/input/iitgai-ovha-23/images/medicakolkata_ccu2_mon--7_2022_5_30_4_6_38.jpeg
/kaggle/input/iitgai-ovha-23/images/medicakolkata_rcu_mon--8_2022_6_20_22_4_31.jpeg
/kaggle/input/iitgai-ovha-23/images/medicakolkata_ccu1_mon--10_2022_6_8_15_1_40.jpeg
/kaggle/input/iitgai-ovha-23/images/medicakolkata_ccu2_mon--2_2022_6_8_7_6_12.jpeg
/kaggle/input/iitgai-ovha-23/images/spsludhiana_micuiithirdfloo

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/iitgai-ovha-23/train_labels.csv')
#print(df)

In [ ]:
print(df.head())

In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [ ]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
def parse_label(label_str):
    label = label_str.strip('[]').split(', ')
    label = [float(coord) for coord in label]
    return np.array(label, dtype=np.float32)
def extract_screen(image_path, label):
    image = cv2.imread(image_path)
    label = label.reshape((4, 2))
    target_coordinates = np.array([[0, 0], [1280, 0], [1280, 720], [0, 720]], dtype=np.float32)
    matrix = cv2.getPerspectiveTransform(label, target_coordinates)
    result = cv2.warpPerspective(image, matrix, (1280, 720))
    return result
labels = pd.read_csv('/kaggle/input/iitgai-ovha-23/train_labels.csv')
labels['points'] = labels['points'].apply(parse_label)
image_paths = labels['image_name'].apply(lambda x: f'/kaggle/input/iitgai-ovha-23/images/{x}')
X = []
y = []

for i, image_path in enumerate(image_paths):
    label = labels.loc[i, 'points']  # Get label as a numpy array
    screen = extract_screen(image_path, label)
    X.append(screen)
    y.append(label)

X = np.array(X)
y = np.array(y)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(720, 1280, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(8)
])
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)


model.compile(optimizer=custom_optimizer, loss='mean_squared_error')
batch_size = 4
num_epochs = 10
cost_history = []

for epoch in range(num_epochs):
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        cost = model.train_on_batch(X_batch, y_batch)
        cost_history.append(cost)

        # Print cost every 100 iterations
        if i % 100 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Iteration {i}/{len(X_train)}, Cost: {cost:.4f}')
sample_submission = pd.read_csv('/kaggle/input/iitgai-ovha-23/sample_submission.csv')

predictions = []
for index, row in sample_submission.iterrows():
 
    image_name = row['image_name']
    label = parse_label(row['points'])

    image_path = f'/kaggle/input/iitgai-ovha-23/test_images/{image_name}'  
   
    screen = extract_screen(image_path, label)
    screen = np.expand_dims(screen, axis=0)  
 
    predicted_points = model.predict(screen)

    predictions.append([image_name] + predicted_points.flatten().tolist())
    
columns = ['image_name'] + [f'point_{i}' for i in range(1, 9)]

predictions_df = pd.DataFrame(predictions, columns=columns)

predictions_df.to_csv('/kaggle/working/result.csv', index=False)


In [ ]:

predictions_df = pd.read_csv('/kaggle/working/result.csv')

predictions_df['points'] = predictions_df.apply(lambda row: [row[f'point_{i}'] for i in range(1, 9)], axis=1)

predictions_df = predictions_df.drop(columns=[f'point_{i}' for i in range(1, 9)])

predictions_df.to_csv('/kaggle/working/submission.csv', index=False)


In [ ]:
import os

current_file_path = '/kaggle/working/result.csv'  # Replace with the correct file path

new_file_path = '/kaggle/working/submission.csv'

os.rename(current_file_path, new_file_path)


In [ ]:
df2 = pd.read_csv('/kaggle/working/submission.csv')

df2['points'] = df2[['point_1', 'point_2', 'point_3', 'point_4', 'point_5', 'point_6', 'point_7', 'point_8']].apply(lambda row: '\t'.join(map(str, row)), axis=1)

df2 = df2.drop(['point_1', 'point_2', 'point_3', 'point_4', 'point_5', 'point_6', 'point_7', 'point_8'], axis=1)

In [ ]:
submission_csv = df2.copy()

In [ ]:
# import cv2
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.applications import VGG16

# # Define a function to parse label string into numpy array
# def parse_label(label_str):
#     label = label_str.strip('[]').split(', ')
#     label = [float(coord) for coord in label]
#     return np.array(label, dtype=np.float32)

# # Define a function to extract screen
# def extract_screen(image_path, label):
#     image = cv2.imread(image_path)
#     label = label.reshape((4, 2))
#     target_coordinates = np.array([[0, 0], [1280, 0], [1280, 720], [0, 720]], dtype=np.float32)
#     matrix = cv2.getPerspectiveTransform(label, target_coordinates)
#     result = cv2.warpPerspective(image, matrix, (1280, 720))
#     return result

# # Load labels and images
# labels = pd.read_csv('/kaggle/input/iitgai-ovha-23/train_labels.csv')
# labels['points'] = labels['points'].apply(parse_label)
# image_paths = labels['image_name'].apply(lambda x: f'/kaggle/input/iitgai-ovha-23/images/{x}')

# # Preprocess images and extract screens
# X = []
# y = []

# for i, image_path in enumerate(image_paths):
#     label = labels.loc[i, 'points']  # Get label as a numpy array
#     screen = extract_screen(image_path, label)
#     X.append(screen)
#     y.append(label)

# X = np.array(X)
# y = np.array(y)

# # Split data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Load the pre-trained VGG16 model (excluding the top layers)
# base_model = VGG16(weights='imagenet', include_top=False, input_shape=(720, 1280, 3))

# # Freeze the layers of the pre-trained model
# for layer in base_model.layers:
#     layer.trainable = False

# # Add your custom layers on top of the pre-trained model
# model = tf.keras.Sequential([
#     base_model,
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(8)
# ])

# # Define a custom optimizer with reduced learning rate
# custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Adjust learning rate as needed

# # Compile the model with the custom optimizer
# model.compile(optimizer=custom_optimizer, loss='mean_squared_error')

# # Implement Mini-batch Gradient Descent
# batch_size = 4
# num_epochs = 20
# cost_history = []

# for epoch in range(num_epochs):
#     for i in range(0, len(X_train), batch_size):
#         X_batch = X_train[i:i+batch_size]
#         y_batch = y_train[i:i+batch_size]
#         cost = model.train_on_batch(X_batch, y_batch)
#         cost_history.append(cost)

#         # Print cost every 100 iterations
#         if i % 100 == 0:
#             print(f'Epoch {epoch+1}/{num_epochs}, Iteration {i}/{len(X_train)}, Cost: {cost:.4f}')

# # Load the sample submission CSV file
# sample_submission = pd.read_csv('/kaggle/input/iitgai-ovha-23/sample_submission.csv')

# # Define a list to store the predictions
# predictions = []

# # Iterate through the sample submission rows
# for index, row in sample_submission.iterrows():
#     # Get the image name and corresponding label
#     image_name = row['image_name']
#     label = parse_label(row['points'])
    
#     # Construct the full image path
#     image_path = f'/kaggle/input/iitgai-ovha-23/test_images/{image_name}'  
    
#     # Extract the screen
#     screen = extract_screen(image_path, label)
#     screen = np.expand_dims(screen, axis=0)  
    
#     # Make predictions using your model
#     predicted_points = model.predict(screen)
    
#     # Append the image name and predicted points to the list
#     predictions.append([image_name] + predicted_points.flatten().tolist())

# # Define the columns for the DataFrame
# columns = ['image_name'] + [f'point_{i}' for i in range(1, 9)]

# # Convert the predictions list to a DataFrame
# predictions_df = pd.DataFrame(predictions, columns=columns)

# # Save the predictions to a CSV file
# predictions_df.to_csv('/kaggle/working/result.csv', index=False)


In [ ]:
# # Load the predictions CSV file
# predictions_df = pd.read_csv('/kaggle/working/result.csv')

# # Combine all the points into a single column
# predictions_df['points'] = predictions_df.apply(lambda row: [row[f'point_{i}'] for i in range(1, 9)], axis=1)

# # Drop the individual point columns
# predictions_df = predictions_df.drop(columns=[f'point_{i}' for i in range(1, 9)])

# # Save the modified DataFrame to a CSV file
# predictions_df.to_csv('/kaggle/working/submission.csv', index=False)
